In [1]:
from PIL import Image
import requests
from io import BytesIO
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random
import math
import imageio
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import glob
import random
import shutil

Matplotlib is building the font cache; this may take a moment.


In [2]:
# get images from pf_ext
labels = os.path.join('Pf_ext_01.json')

count = 1
phase_count = {
    "train": 1,
    "test": 1
}
phase_count["train"] = 1
phase_count["test"] = 1

phase = "train"
# degs = [45,90,135,180,225,270,315]
degs = [90,180,270,360,450,540,630]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "2":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "1":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1
            for deg in degs:
                name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (((x2-x1)/2)+x1)/aug_w
                    bb_yc = (((y2-y1)/2)+y1)/aug_h
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    bboxs = bboxs[1:]
                    im_aug = Image.fromarray(image_aug)
                    im_aug.save(os.path.join('../data/custom/images',name+".jpg"))
                    f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
                    f.write(bboxs)
                    f.close()
                    print("{} {}".format(phase_count[phase],deg))
                    phase_count[phase] += 1

1 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015250.jpg
pf-train-0000000001
2 90
3 180
4 270
5 360
6 450
7 540
8 630
2 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015251.jpg
pf-train-0000000009
10 90
11 180
12 270
13 360
14 450
15 540
16 630
3 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015252.jpg
pf-train-0000000017
18 90
19 180
20 270
21 360
22 450
23 540
24 630
4 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015253.jpg
pf-train-0000000025
26 90
27 180
28 270
29 360
30 450
31 540
32 630
5 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015254.jpg
pf-train-0000000033
34 90
35 180
36 270
37 360
38 450
39 540
40 630
6 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/tmp_images/0000015255.jpg
pf-train-0000000041
42 90
43 180
44 270
45 360
46 450
47 540
48 630
7 https://raw.githubusercontent.com/RTANC/malaria_dat

In [3]:
print(phase_count["train"])

936


In [5]:
# get image from pf_01
labels = os.path.join('pf_01.json')
count = 1
# degs = [45,90,135,180,225,270,315]
with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1
            for deg in degs:
                name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
                image_aug, bbs_aug = iaa.Affine(rotate=deg,translate_percent={"x": (-0.25, 0.25), "y": (-0.25, 0.25)},mode="reflect")(image=img, bounding_boxes=bbs)
                aug_h, aug_w, aug_c = image_aug.shape
                clip_off_bbs = bbs_aug.remove_out_of_image(fully=True,partly=True).clip_out_of_image()
                bboxs = ""
                for bb in clip_off_bbs.bounding_boxes:
                    x1 = bb.x1
                    y1 = bb.y1
                    x2 = bb.x2
                    y2 = bb.y2
                    bb_w = (x2-x1)/aug_w
                    bb_h = (y2-y1)/aug_h
                    bb_xc = (x1/aug_w) + (bb_w/2)
                    bb_yc = (y1/aug_h) + (bb_h/2)
                    bboxs += "\n"+ str(bb.label) + " " + str(bb_xc) + " " + str(bb_yc) + " " + str(bb_w) + " " + str(bb_h)
                if len(clip_off_bbs.bounding_boxes) > 0:
                    bboxs = bboxs[1:]
                    im_aug = Image.fromarray(image_aug)
                    im_aug.save(os.path.join('../data/custom/images',name+".jpg"))
                    f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
                    f.write(bboxs)
                    f.close()
                    print("{} {}".format(deg,name))
                    phase_count[phase] += 1

1 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_01/Data20200115-01_Wholeslide_Default_Extended_i01j17.jpg
90 pf-train-0000000937
180 pf-train-0000000938
270 pf-train-0000000939
360 pf-train-0000000940
450 pf-train-0000000941
540 pf-train-0000000942
2 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_01/Data20200115-01_Wholeslide_Default_Extended_i01j21.jpg
270 pf-train-0000000944
3 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_01/Data20200115-01_Wholeslide_Default_Extended_i01j22.jpg
4 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_01/Data20200115-01_Wholeslide_Default_Extended_i02j15.jpg
90 pf-train-0000000947
180 pf-train-0000000948
270 pf-train-0000000949
360 pf-train-0000000950
450 pf-train-0000000951
540 pf-train-0000000952
630 pf-train-0000000953
5 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_01/Data20200115-01_Wholeslide_Default_E

In [6]:
print(phase_count["train"])

6188


In [7]:
# get image from pf_test
labels = os.path.join('pf_test.json')

count = 1
phase_count = {
    "test": 1
}

phase = "test"
degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
            rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "1":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
                    rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            img = np.array(img)
            bbs = BoundingBoxesOnImage(rects,shape=img.shape)
            count += 1
            phase_count[phase] += 1

1 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/16.jpg
pf-test-0000000001
2 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/23.jpg
pf-test-0000000002
3 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/3.jpg
pf-test-0000000003
4 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/31.jpg
pf-test-0000000004
5 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/6.jpg
pf-test-0000000005
6 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/60.jpg
pf-test-0000000006
7 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/63.jpg
pf-test-0000000007
8 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/66.jpg
pf-test-0000000008
9 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_test/68.jpg
pf-test-0000000009
10 https://r

In [8]:
# get image from pf_02 (Test set)
labels = os.path.join('pf_02.json')

count = 1
phase_count = {
    "test": 11
}

phase = "test"
degs = [45,90,135,180,225,270,315]

with open(labels,'r') as json_file:
    data = json.load(json_file)
    images = data["_via_img_metadata"]
    for key in images:
        url = images[key]["filename"]
        leng = len(images[key]["regions"])
        bboxs = ""
        if leng > 0 and count <= 100:
            name = "pf-" + phase + "-{:010d}".format(phase_count[phase])
            print("{} {}".format(count,url))
            print(name)
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
            w, h = img.size
            img.save(os.path.join('../data/custom/images',name+".jpg"))
            f = open(os.path.join('../data/custom/labels',name+".txt"),'w')
#             rects = []
            for box in images[key]["regions"]:
                b = box["shape_attributes"]
                x1 = b["x"]/w
                y1 = b["y"]/h
                bw = b["width"]/w
                bh = b["height"]/h
                xc = x1 + (bw/2)
                yc = y1 + (bh/2)
                lbl = box["region_attributes"]["type"]
                if lbl == "2":
                    lbl = "1"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
#                     rects.append(BoundingBox(x1=b["x"],y1=b["y"],x2=(b["x"] + b["width"]),y2=(b["y"] + b["height"]),label=lbl))
                elif lbl == "0":
                    lbl = "0"
                    bboxs += "\n"+ lbl + " " + str(xc) + " " + str(yc) + " " + str(bw) + " " + str(bh)
            bboxs = bboxs[1:]
            f.write(bboxs)
            f.close()
            phase_count[phase] += 1
#             img = np.array(img)
#             bbs = BoundingBoxesOnImage(rects,shape=img.shape)
        count += 1

2 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_02/Data20200115-02_Wholeslide_Default_Extended_i01j02.jpg
pf-test-0000000011
4 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_02/Data20200115-02_Wholeslide_Default_Extended_i01j04.jpg
pf-test-0000000012
5 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_02/Data20200115-02_Wholeslide_Default_Extended_i01j05.jpg
pf-test-0000000013
6 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_02/Data20200115-02_Wholeslide_Default_Extended_i01j06.jpg
pf-test-0000000014
7 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_02/Data20200115-02_Wholeslide_Default_Extended_i01j07.jpg
pf-test-0000000015
8 https://raw.githubusercontent.com/RTANC/malaria_dataset/master/Falciparum/pf_02/Data20200115-02_Wholeslide_Default_Extended_i01j08.jpg
pf-test-0000000016
9 https://raw.githubusercontent.com/RTANC/malaria_dataset/master

In [ ]:
# summary training set
# files = glob.glob("images/pf-train-*.jpg")
# count = 0
# count_class = [0,0]

# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     lines = list(f)
#     f.close()
#     print(img)
#     for line in lines:
#         print(line)
#         class_id, xc, yc, wb, hb = line.split()
#         count_class[int(class_id)] += 1
#     count += 1
# print(count)
# print(count_class)

In [ ]:
# clear empty box
# files = glob.glob("images/*")
# count = 0
# for file in files:
#     img = file.split("/")[-1]
#     name = img.split('.')[0]
#     f = open("labels/"+name+".txt","r")
#     txt = list(f)
#     f.close()
#     if len(txt) == 0:
#         print(img)
#         os.remove("labels/"+name+".txt")
#         os.remove(file)
#         count += 1
# print(count)

In [ ]:
# labels = os.path.join('Pf','pf_01.json')
# count = 0
# with open(labels,'r') as json_file:
#     data = json.load(json_file)
#     images = data["_via_img_metadata"]
#     for key in images:
#         url = images[key]["filename"]
#         if count >= 120 and count <= 240:
#             name = "{:010d}".format(count)
#             print("{} {}".format(count,url))
#             response = requests.get(url)
#             img = Image.open(BytesIO(response.content))
#             w, h = img.size
#             img.save(os.path.join('data','samples',name+".jpg"))
#         count += 1
    

In [ ]:
# count = 9784
# for i in range(count,12583):
#     name = "{:010d}".format(i)
#     print(name)
#     os.remove(os.path.join("data/custom/images/",name+".jpg"))
#     os.remove(os.path.join("data/custom/labels/",name+".txt"))